In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import functools
import json
import os
import re

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 400)
pd.options.display.max_rows = 100

In [3]:
# path
DATA_PATH = "../data/"
RESULT_PATH = "../result/"
MASTER_PATH = "../master/"

In [4]:
# json定義
with open(os.path.join(MASTER_PATH, "therapeutic_category_code_master.json"), encoding="utf-8") as f:
    therapeutic_category_code_dict = json.load(f)

with open(os.path.join(MASTER_PATH, "drug_rename_columns.json"), encoding="utf-8") as f:
    drug_rename_columns_dict = json.load(f)

In [5]:
# single table
def read_table(table_name: str) -> pd.DataFrame:
    """各excelファイルを読み込む
    Args:
        table_name (str):テーブル名称

    Returns:
        pd.DataFrame: 処理後のdf
    """
    df= pd.read_excel(DATA_PATH + table_name + '.xlsx')
    return df

def rename_column(df: pd.DataFrame, table_column_dict: dict)->pd.DataFrame:
    """列名を変更する。"""
    return df.rename(columns=table_column_dict)

In [6]:
# 薬剤リストの読み込みと列名処理
drug = read_table(table_name="drug_list_2025")

In [7]:
def generate_deletion_patterns(spec: str) -> list:
    """specification の中から削除対象となるすべての部分列を生成（2文字以上）"""
    tokens = list(spec)
    patterns = set()
    for i in range(len(tokens)):
        for j in range(i + 2, len(tokens) + 1): 
            sub = ''.join(tokens[i:j])
            patterns.add(sub)
    return sorted(patterns, key=len, reverse=True)

def remove_bracketed_phrases(text: str) -> str:
    """全角「」()で囲まれた部分をすべて削除"""
    text = re.sub(r'「[^」]*」', '', text)
    text = re.sub(r'（[^）]*）', '', text)
    text = re.sub(r'〔[^〕]*〕', '', text)
    text = re.sub(r'〈[^〉]*〉', '', text)
    return text.strip()

def remove_keywords(text: str) -> str:
    """特定の語で drug_name が終わる場合のみ削除"""
    keywords = ["ＯＤ錠", "ＯＤ錠ＴＲＥ", "ＯＤフィルム", "ＳＲ錠", "口腔内崩壊錠", "注射液", "細粒", 
                "ドライシロップ", "散剤", "錠剤", "カプセル", "軟膏", "クリーム", "液剤", "シロップ", 
                "坐剤", "貼付剤", "吸入剤", "ｇ", "ｍｇ", "ｍＬ", "単位", "カプセル剤", "軟膏剤", "クリーム剤", 
                "シロップ剤", "坐剤剤", "吸入剤剤", "散", "徐放顆粒", "本末", "ＣＲ錠", "ＣＲカプセル",
                "徐放錠", "徐放カプセル", "徐放剤", "顆粒", "ＤＳ", "ＤＳ錠", "ＤＳカプセル", "ＤＳ剤",
                "ＤＳ小児用", "小児用", "内用液分包", "ｇ／", "内用液", "ＬＡ錠ＭＩ", "ＬＡ錠",
                "原末", "原末剤", "原末カプセル", "原末錠", "原末散", "原末顆粒", "原末小児用",
                "原末小児用カプセル", "原末小児用錠", "ドライシロップ分包", "内用液ｇ分包", "配合錠",
                "配合剤", "配合カプセル", "配合錠剤", "配合散剤", "配合顆粒", "配合小児用", "糖衣錠",
                "配合錠ＬＤ", "配合錠ＡＰ", "配合錠ＢＰ", "配合錠ＨＤ", "ペン", "点眼液", "皮下注", 
                "静注", "点滴静注", "配合点眼液", "配合懸濁性点眼液", "皮下注シリンジ", "静注用", "輸液",
                "内服ゼリー", "舌下錠", "ゼリー", "注射用", "ＥＮ錠", "注用", "注用シリンジ", "ワコー", 
                "シオエ", "注シリンジ", "点滴", "注", "顆粒分包", "静注１シリンジ", "μ", "配合",
                "徐放性", "徐放", "内服ゼリー分包", "分包", "懸濁用", "坐薬", "膣錠", "泌尿器科用灌流液",
                "点鼻液", "膣用", "注腸", "テープ", "点鼻スプレー", "ＳＴ錠口腔用", "トローチ",
                "口腔用", "外用", "浣腸用", "浣腸液", "浣腸", "ＸＲ錠", "粉末", "ＢＰ錠", "ＬＤ錠",
                "ＨＤ錠", "ＡＰ錠", "ＸＲカプセル", "ＸＲ剤", "ＸＲ錠剤", "ＸＲ顆粒", "ＸＲ小児用", 
                "吸入１００回", "吸入２００回", "吸入５０回", "吸入用", "吸入粉末", "吸入カプセル",
                "吸入", "軟膏", "回", "吸入液", "μｇ", "μｇ／回", "μｇ／ｍＬ", "μｇ／日", "用",
                "吸入用低用量", "吸入用中用量", "吸入用高用量", "消毒液", "消毒用", "消毒剤", "静"
                "消毒", "消毒用液", "消毒用スプレー", "外用液", "消毒Ｂ液", "消毒Ｃ液", "細粒錠", 
                "崩壊錠", "錠", "１日用", "３日用", "シリンジ", "筋注", "点滴", "筋", "配合", 
                "内服液", "顆粒", "速崩", "微粒", "未", "液", "腸溶", "腸溶性", "医療", "内用剤",
                "配合内用剤", "徐放", "静注用専用溶解", "専用溶解", "点滴静注バッグ", "キット", "静注液バッグ",
                "吸入懸濁用セット", "吸入粉末剤", "鼻腔", "組織接着", "外用スプレー", "外用泡状スプレー", 
                "４μｇ／ｃ㎡", "８μｇ／ｃ㎡", "１２μｇ／ｃ㎡", "１６μｇ／ｃ㎡", "２０μｇ／ｃ㎡", "膣", "点鼻粉末剤",
                "５千", "１万", "２万", "３万", "４万", "５万", "６万", "７万", "８万", "９万", "１０万", "１番",
                "２番", "３番", "４番", "５番", "６番", "７番", "８番", "９番", "１０番", "アテオス", "・", 
                "フレックスタッチ", "ソロスター", "カート", "ミリオ", "ペンフィル", "イノレ", "フレックス"]
    keywords = sorted(keywords, key=len, reverse=True)
    text = text.strip()
    for kw in keywords:
        if text.endswith(kw):
            return text[: -len(kw)].strip()
    return text

def remove_trailing_zenkaku(text: str) -> str:
    """
    全角数字または全角記号が末尾にある場合、それを削除
    """
    return re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ．\.／／＋\-\%％＊!！＠＃＄＆＝～]+$', '', text).strip()

def clean_drug_name(row):
    name = str(row["drug_name"])
    spec = str(row["specification"])
    for pattern in generate_deletion_patterns(spec):
        name = name.replace(pattern, "")
    name = remove_bracketed_phrases(name)
    name = remove_trailing_zenkaku(name)
    name = remove_keywords(name)
    name = remove_trailing_zenkaku(name)
    name = remove_keywords(name)
    return name.strip()

def transform_drug(df: pd.DataFrame) -> pd.DataFrame:
    # 列名の変更
    df = df.rename(columns=drug_rename_columns_dict["drug"])
    # 列の選択
    df = df[["drug_price_standard_code", "ingredient_name", "specification", "drug_name"]]
    # 規格列から()で囲まれた部分を削除
    df["specification"] = df["specification"].apply(remove_bracketed_phrases)
    # 薬効分類列の追加
    df["therapeutic_category_code"] = df["drug_price_standard_code"].astype(str).str[:4]
    # 薬効分類+成分
    df["therapeutic_category_ingredient_code"] = df["drug_price_standard_code"].astype(str).str[:7]
    # 一般名の標準化
    df["drug_name_ex"] = df.apply(clean_drug_name, axis=1)
    return df

In [8]:
DRUG = transform_drug(drug)

DRUG

,drug_price_standard_code,ingredient_name,specification,drug_name,therapeutic_category_code,therapeutic_category_ingredient_code,drug_name_ex
0,1121001X1018,ブロモバレリル尿素,１ｇ,ブロモバレリル尿素,1121,1121001,ブロモバレリル尿素
1,1123001X1012,抱水クロラール,１ｇ,抱水クロラール,1123,1123001,抱水クロラール
2,1124001B1039,エスタゾラム,１％１ｇ,ユーロジン散１％,1124,1124001,ユーロジン
3,1124001F1022,エスタゾラム,１ｍｇ１錠,ユーロジン１ｍｇ錠,1124,1124001,ユーロジン
4,1124001F1030,エスタゾラム,１ｍｇ１錠,エスタゾラム錠１ｍｇ「アメル」,1124,1124001,エスタゾラム
...,...,...,...,...,...,...,...
13149,8219701S5020,フェンタニルクエン酸塩,８ｍｇ１枚,フェントステープ８ｍｇ,8219,8219701,フェントス
13150,8219701S5039,フェンタニルクエン酸塩,８ｍｇ１枚,フェンタニルクエン酸塩１日用テープ８ｍｇ「第一三共」,8219,8219701,フェンタニルクエン酸塩
13151,8219701S5047,フェンタニルクエン酸塩,８ｍｇ１枚,フェンタニルクエン酸塩１日用テープ８ｍｇ「テイコク」,8219,8219701,フェンタニルクエン酸塩
13152,8219701S6027,フェンタニルクエン酸塩,０．５ｍｇ１枚,フェントステープ０．５ｍｇ,8219,8219701,フェントス


In [10]:
output_file = os.path.join(RESULT_PATH, "DRUG_cleaned.xlsx")
# Excelに保存（index=False で行番号は出力しない）
DRUG.to_excel(output_file, index=False)